In [1]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pandas as pd
# Custom utils
from utils.simulator.simulator import MCSimulation
# Tf imports
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

In [2]:
def custom_derivative_model(x, model, multioutput = True, mode = 'centered'):
    h = 1e-1
    # Size x
    x_dim, y_dim = x.shape
    # Gradient vector
    if multioutput:
        gradient = np.zeros((x_dim, y_dim, y_dim))
    else:
        gradient = np.zeros((x_dim, y_dim, 1))
    for i in range(y_dim):
        # Vector for partial derivative estimation
        offset_tensor = np.zeros((x_dim, y_dim))
        offset_tensor[:, i] = h
        offset_tensor = tf.convert_to_tensor(offset_tensor,
                                             dtype = tf.float32)
        # Constantes:
        denominator = h
        if mode == 'progressive':
            numerator = model(
                tf.math.add(x, offset_tensor)
            ) - model(
                x
            ) 
        elif mode == 'regressive':
            numerator = model(
                x
            ) - model(
                tf.math.subtract(x, offset_tensor)
            )
        elif mode == 'centered':
            numerator = tf.math.subtract(
                model(
                    tf.math.add(x, offset_tensor)
                ), model(
                    tf.math.subtract(x, offset_tensor)
                )
            )
        denominator = 2 * h
        gradient [:, i, :] = numerator / denominator
    gradient = tf.convert_to_tensor(gradient,
                                        dtype = tf.float32)
    return gradient

In [29]:

input_layer = keras.Input(shape = (2,), name='input_nn')
'''gru_unit = layers.GRU(
    128, 
    name = 'sequential_layer'
)(input_layer)'''
dense_unit = layers.Dense(
    units = 64,
    name = 'dense_layer'
)(input_layer)
output_layer = layers.Dense(
    units = 2, 
    activation = 'relu', 
    name = 'first_dense'
)(dense_unit)
custom_model = keras.Model(
    inputs=[input_layer],
    outputs=[output_layer],
    name = 'test_model'
)

In [30]:
# Domain -5:5
x = tf.convert_to_tensor(
    np.array(range(-10,10)).reshape((10,2)),
    dtype = tf.float32    
)
# Test 
xs = tf.Variable(x, trainable = True, name = 'x')
with tf.GradientTape() as tape, tf.GradientTape() as tape_2, tf.GradientTape() as tape_3:
    tape.watch(xs)
    tape_2.watch(xs)
    tape_3.watch(xs)
    y = custom_model(xs)
# This represents dV/dX
grads = tape.gradient(y, {
    'x':xs
})
jacobian = tape_2.jacobian(y, {
    'x':xs
})

In [18]:
custom_grads = tf.transpose(custom_derivative_model(x, custom_model))

ValueError: could not broadcast input array from shape (2,10) into shape (10,)

In [43]:
jacobian_reshaped = tf.reshape(jacobian['x'], (10, 10, 2, 2))
jacobian_reshaped.shape, jacobian_reshaped[6]

(TensorShape([10, 10, 2, 2]),
 <tf.Tensor: shape=(10, 2, 2), dtype=float32, numpy=
 array([[[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],
 
        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],
 
        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],
 
        [[ 0.1675716 , -0.06344406],
         [ 0.        ,  0.        ]],
 
        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],
 
        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],
 
        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],
 
        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],
 
        [[ 0.20841727,  0.5425839 ],
         [ 0.        ,  0.        ]],
 
        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ]]], dtype=float32)>)

In [20]:
tf.linalg.diag_part(jacobian['x'])

<tf.Tensor: shape=(2, 10, 2), dtype=float32, numpy=
array([[[-0.3357237 ,  0.        ],
        [ 0.        ,  0.        ],
        [-0.30415174,  0.        ],
        [ 0.        ,  0.        ],
        [-0.00158158,  0.        ],
        [-0.16155705,  0.        ],
        [-0.02066748,  0.        ],
        [ 0.        ,  0.        ],
        [-0.32954675,  0.        ],
        [-0.21719304,  0.        ]],

       [[ 0.        ,  0.3730056 ],
        [ 0.        , -0.2060039 ],
        [ 0.        , -0.25046194],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.        ],
        [ 0.        ,  0.36052293],
        [ 0.        , -0.29172918],
        [ 0.        , -0.12877887]]], dtype=float32)>

In [15]:
grads['x'].shape

TensorShape([1, 10])

In [ ]:
grads['x'].T, custom_grads, jacobian['x']

(<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
 array([[-0.5670204 , -1.0382694 , -0.7536539 ,  0.1433045 ,  0.91822445,
         -0.33055848, -0.03884544, -0.08828928, -0.5248461 ,  1.4366901 ]],
       dtype=float32)>,
 <tf.Tensor: shape=(10, 10), dtype=float32, numpy=
 array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [-0.06004572, -0.31398535, -0.13197541, -0.10521173,  0.13706803,
         -0.03125548,  0.17498016,  0.05945802, -0.02276778,  0.22522807],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [-0.16636491, -0.15045166,  0.13496935, -0.18328667,  0.0385952 ,
         -0.00757515, -0.04257083, -0.18448412, -0.2458